## Shopee Code League 2020
<h3>Order Brushing Challenge</h3>
<b>Team: DragonSnake</b>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta 
sns.set()

In [2]:
%%capture
from tqdm import tqdm
tqdm().pandas()

<h2>Explore Dataset</h2>

In [3]:
df = pd.read_csv("./data/order_brush_order.csv")

Let's take a glimpse of the dataset

In [4]:
df.head(5)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [5]:
(df.shape[0], df.shape[1])

(222750, 4)

The dataset contains 222750 rows and 4 columns (orderid, shopid, userid, event_time)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   orderid     222750 non-null  int64 
 1   shopid      222750 non-null  int64 
 2   userid      222750 non-null  int64 
 3   event_time  222750 non-null  object
dtypes: int64(3), object(1)
memory usage: 6.8+ MB


We can see that the event_time columns is still in string (object) type, thus, conversion to datetime for later processing is essential !

In [7]:
df['event_time'] = pd.to_datetime(df['event_time'])

In [8]:
# Check data type after converting whether it has been converted to Timestamp (pandas)
df['event_time'].dtype

dtype('<M8[ns]')

<h2>Rules</h2>

The dataframe consists of 4 columns which are order's ID, shop's ID, user's ID and the time when the order event occured.

<b>Rules defined by Shopee</b>
<li>Concentrate rate = Number of orders within 1 hour / Number of unique buyers within 1 hour (per shopid)</li>
If concentrate rate is equal to 3 or greater -> Shop would have deemed to do order brushing
<li>Buyer proportion = number of individual's orders / total orders within 1 hour</li>

<h3>Dataset Details</h3>

In [9]:
df['shopid'].value_counts()

147941492    11703
61556313      6691
54615708      2640
91799978       707
57189823       687
             ...  
158235164        1
147997211        1
166898182        1
194982850        1
189530112        1
Name: shopid, Length: 18770, dtype: int64

There are 18770 shop's IDs that we need to investigate whether each may perform order brushing cheat.

Let's convert unique shop's id to list !

In [10]:
shop_list = df['shopid'].unique().tolist() # Convert numpy array to python list

Let's try to findout the number of orders by shop's id

In [11]:
def total_orders_by_shop_id(df, shop_list):
    shop_orders = {}
    for shop_id in tqdm(shop_list):
        df_loc = df.loc[(df['shopid'] == shop_id)]
        shop_orders[shop_id] = df_loc.shape[0]
    return shop_orders

According to the rules of shopee, concentrate rate per shop must be equal or greater than 3 in order to be in the suspicious of order brushing.
Therefore, we must filter out list of shops that have 3 or more orders.
Let's create a filtered dictionary !

In [12]:
%%time
# Output number of orders by shop id
shop_orders = total_orders_by_shop_id(df, shop_list)

100%|██████████| 18770/18770 [00:12<00:00, 1492.57it/s]

Wall time: 12.6 s


In [13]:
# Filter dictionary for shop with 3 or more orders
filtered_shops = {k:v for k,v in shop_orders.items() if v >= 3}

Based on the filtered shops, we can start filtering out the dataframe directly

In [14]:
# Filter list of shops that have 3 or more orders
df = df.loc[df['shopid'].isin(filtered_shops.keys())]
df.shape

(210357, 4)

There are 210357 orders left after filtering order's ID of shops that have under 3 orders 

In [15]:
df = df.sort_values(by=['shopid', 'event_time']).reset_index(drop=True)

<b>Rearrange Dataframe</b>
<p>In order to handle processing orders of each shop easily following a time series, we have sorted the dataframe by shopid and event_time in ascending.</p>

Let's try to save user orders per shop id under user's id and event_time data format in 2d array-list

In [16]:
%%time
user_orders_by_shop_id = {}

for index, row in df.iterrows():
    current_shop_id = row['shopid']
    current_user_id = row['userid']
    current_order_time = row['event_time']
    if current_shop_id not in user_orders_by_shop_id:
        new_list = []
        new_dict = {}
        new_dict['userid'] = current_user_id
        new_dict['event_time'] = current_order_time
        new_list.append(new_dict)
        user_orders_by_shop_id[current_shop_id] = new_list
    else:
        new_dict = {}
        new_dict['userid'] = current_user_id
        new_dict['event_time'] = current_order_time
        user_orders_by_shop_id[current_shop_id].append(new_dict)

Wall time: 13.1 s


Let's define function to calculate concentration rate for each shop id according to event time.
<p>
    <b>Notes:</b> It is much faster to use pd.DataFrame.at function to query by given index than iloc. (Great reduction in time complexity using .at)
</p>

In [17]:
%%time
# Define shop dictionary including list of orders within 1 hour
shop_dict = {}

# Loop through shop's orders dictionary by shop ID
for shopid, orders in user_orders_by_shop_id.items():
    current_shop_intervals = []
    #Loop through order details in each shop's id
    for index in range(len(orders)):
        # Get current order time 
        current_order_time = orders[index]['event_time']
        # Reset periods/intervals after each time window (1hour)
        one_hour_period = []
        one_hour_period.append(orders[index])
        for order_index in range(index + 1, len(orders)):
            if(index == order_index):
                print("true")
            next_order_time = orders[order_index]['event_time']
            time_between = (next_order_time - current_order_time).total_seconds() / 3600.0
            if (time_between <= 1):
                one_hour_period.append(orders[order_index])
            else:
                break
        if(len(one_hour_period) >= 3):
            current_shop_intervals.append(one_hour_period)
    shop_dict[shopid] = current_shop_intervals


Wall time: 20.9 s


In [18]:
len(shop_dict)

9031

In [19]:
%%time
ans_shops = {}

for shopid, intervals in shop_dict.items():
    brushing_users = []
    for periods in intervals:
        concentrate_rate = 0;
        unique_user_orders = {}
        for order in periods:
            if (order['userid'] not in unique_user_orders):
                unique_user_orders[order['userid']] = 1/len(periods)
            else:
                unique_user_orders[order['userid']] += 1/len(periods)
        concentrate_rate = len(periods) / len(unique_user_orders)
        # Check if shop deemed to have order brushing
        if (concentrate_rate >= 3):
            max_value = max(unique_user_orders.values())
            sus_users = [k for k,v in unique_user_orders.items() if v == max_value]
            for sus_user in sus_users:
                if (sus_user not in brushing_users):
                    brushing_users += sus_users

    ans_shops[shopid] = brushing_users

Wall time: 962 ms


In [20]:
# Create list of shop ids and users id as answers to the problem
ans_shop_id = [k for k,v in ans_shops.items()]
ans_shop_users = [v for k,v in ans_shops.items()]

In [21]:
# Concat-ing order-brushed users for submission
ans_users = []
for users in ans_shop_users:
    ans_string = ""
    if (len(users) >= 1):
        for user in users:
            if(len(ans_string) == 0):
                ans_string += str(user)
            else:
                ans_string += "&" + str(user)
    else:
        ans_string = "0"
    ans_users.append(ans_string)

Create a answer dictionary including shop id and user ids

In [22]:
# Create dictionary with shop id as key and suspicious user as value
ans_dict = dict(zip(ans_shop_id, ans_users))
# Create dataframe from answer dictionary above
ans_df = pd.DataFrame(ans_dict.items(), columns=['shopid', 'userid'])
ans_df.head()

,shopid,userid
0,10061,0
1,10084,0
2,10100,0
3,10132,0
4,10133,0


In [23]:
# Create numpy array of all shops
shop_array = np.array(shop_list)
# Reshape shop's ids array into (x, y) rows/columns format
shop_array_data = shop_array.reshape(len(shop_array), 1)
# Create dataframe from reshaped array
shop_array_df = pd.DataFrame({'shopid': shop_array_data[:,0]})

Let's merge answered shops to list of all shops

In [24]:
merge_df = shop_array_df.merge(ans_df, how='left', on='shopid')
merge_df.fillna("0", inplace=True)
merge_df = merge_df.sort_values(by=['shopid']).reset_index(drop=True)

In [25]:
# Output answer to csv file
merge_df.to_csv("out.csv", index=False)

<h2>Legit check to top submissions</h2>
Pending confirmed results (?)

In [26]:
correct_df = pd.read_csv("data/prediction.csv")
correct_df = correct_df.sort_values(by=['shopid']).reset_index(drop=True)
correct_df.head()

,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0


In [27]:
def validation(list_1,list_2):
    return sorted(list_1) == sorted(list_2)

In [28]:
count_diff_0 = 0
count_diff_arr = 0
for i in range(merge_df.shape[0]):
    if(len(merge_df.at[i, 'userid']) > 1):
        ans_me_array = merge_df.at[i, 'userid'].split("&")
        ans_top_array = correct_df.at[i, 'userid'].split("&")
        if (validation(ans_me_array, ans_top_array) == False):
            print("Different submission at index {} with userids are: {} compared to: {}".format(i, merge_df.at[i, 'userid'], correct_df.at[i, 'userid']))
            count_diff_arr += 1
    else:
        if (merge_df.at[i, 'userid'] != correct_df.at[i, 'userid']):
            print("Different submission at index {} with userids are: {} compared to: {}".format(i, merge_df.at[i, 'userid'], correct_df.at[i, 'userid']))
            count_diff_0 += 1
print("\nTotal number of difference compared to top answers: {} (zero answer) and {} (with userid answer)".format(count_diff_0, count_diff_arr))
proportion_score = (407.275 - ((count_diff_0 * 0.005) + (count_diff_arr * 1))) / 407.275
print("Possible score on kaggle submission: ~{:.4f}".format(proportion_score))

Different submission at index 13 with userids are: 0 compared to: 214988798
Different submission at index 332 with userids are: 0 compared to: 141006168
Different submission at index 836 with userids are: 0 compared to: 740844
Different submission at index 990 with userids are: 162508227&13135622&137245836&215382704 compared to: 215382704
Different submission at index 1003 with userids are: 0 compared to: 197569856
Different submission at index 1019 with userids are: 0 compared to: 148176353
Different submission at index 1028 with userids are: 0 compared to: 84811421
Different submission at index 1398 with userids are: 0 compared to: 128702876
Different submission at index 1526 with userids are: 0 compared to: 192684666
Different submission at index 2193 with userids are: 0 compared to: 114498557
Different submission at index 2204 with userids are: 0 compared to: 33794624
Different submission at index 2764 with userids are: 0 compared to: 214226569
Different submission at index 3194 wi

<h2>## End of Notebook ##</h2>
<b>Member: Nguyen Pham Quoc Minh</b>
<p><b>Team: DragonSnake</b></p>